In [1]:
import numpy as np
import tensorflow as tf

In [2]:
start = tf.constant([1.548837512, 0.07374419285, -0.009645789198, -2.307796117, 0.03278595457, 2.367094903, 0.7636861643], dtype=tf.float32)
goal = tf.constant([0.5892901589, 0.4606513663, 0.02096505987, -2.409281471, -1.05241235, 1.91614986, 0.5328634008], dtype=tf.float32)

In [3]:
blue =    tf.constant([0.5892901589, 0.4606513663, 0.02096505987, -2.409281471, -1.05241235, 1.91614986, 0.5328634008], dtype=tf.float32)
green =   tf.constant([0.04828069357, 0.321198491, -0.09149035608, -2.445509451, 0.0339262541, 2.766537249, -0.7381757918], dtype=tf.float32)
red =     tf.constant([0.4973887075, 0.3626231152, -0.1262784644, -2.353311054, 0.1667072496, 2.697354126, -0.9114452175], dtype=tf.float32)

In [4]:
min_angles = [-2.8973, -1.7628, -2.8973, -3.0718, -2.8973, -0.0175, -2.8973]
max_angles = [2.8973, 1.7628, 2.8973, -0.0698, 2.8973, 3.7525, 2.8973]

In [5]:
@tf.function
def constrain_start_goal(keyframes, start, goal):
    return tf.concat((start[None, :], keyframes[1:-1], goal[None, :]), 0)

In [6]:
def constraint_to_joint_space(keyframes, min_coords, max_coords):
    return tf.clip_by_value(keyframes, min_coords, max_coords)

In [7]:
@tf.function
def integrate(a, b, function, num_samples=100):
    # from https://en.wikipedia.org/wiki/Numerical_integration
    positions = tf.cast(a + tf.range(num_samples+1)*(b-a)/num_samples, tf.float32)
    values = function(positions)
    integral = (b-a)/num_samples * (values[0]/2 + tf.reduce_sum(values[1:-1], axis=0) + values[-1]/2)
    return integral

In [8]:
@tf.function
def trajectory(t, keyframes):
    """Compute the position of the robot at a given time

    Given a tensor of times t (each element within [0, 1]) and a trajectory in
    joint space given by a set of keyframes, compute the pose of the robot
    for each time t.

    All dimensions of the time tensor are assumed to be batch dimensions.

    The trajectory is assumed to be piece-wise linear between keyframes. At time
    t=0 the robot is in the position of the first keyframe, and at time t=1 the robot
    is in the pose of the last keyframe. The last dimension of a keyframe indicates 
    the pose, whereas the first dimension indicates the sequence in which the 
    keyframes are being visited. All other dimensions are batch dimensions.

    t - a tensor of any shape with values between [0, 1]
    keyframes - a tensor of keyframes where keyframes[0] denotes the sequence of keyframes
        and keyframes[..., :] denotes the pose of the robot.
    
    """
    N = tf.constant(keyframes.shape[-2], dtype=np.float32)
    direction = tf.concat((keyframes, keyframes[-1, ...][None, ...]), axis=0)[1:, ...] - keyframes
    idx = tf.cast(tf.floor(t * (N-1)), tf.int64)
    time_shape = (*t.shape, *(1 for _ in range(len(keyframes.shape)-1)))
    return tf.gather(direction, idx, axis=0) * tf.reshape(tf.math.floormod(t*N, 1), time_shape) + tf.gather(keyframes, idx, axis=0)

@tf.function
def velocity(t, keyframes):
    N = t.shape[0]
    with tf.GradientTape() as tape:
        tape.watch(t)
        trajectory_points = trajectory(t, keyframes)
    jacobian = tape.jacobian(trajectory_points, t)
    vel = tf.reduce_sum(tf.reshape(jacobian, (*trajectory_points.shape, -1)), axis=-1)
    return vel

@tf.function
def velocity_fast(t, keyframes):
    N = tf.constant(keyframes.shape[0], dtype=np.float32)
    direction = tf.concat((keyframes, keyframes[-1, ...][None, ...]), axis=0)[1:, ...] - keyframes
    idx = tf.cast(tf.floor(t * (N-1)), tf.int64)
    return tf.gather(direction, idx, axis=0) * N

In [9]:
@tf.function
def cost(keyframes):
    return integrate(
        0, 1, 
        lambda t: tf.reduce_sum(velocity_fast(t, keyframes)*velocity_fast(t, keyframes), axis=-1)
    )

def V(start, goal, *, num_keys=10, steps=10):
    num_keys = 10
    steps=10
    keyframes = tf.Variable(
        tf.linspace(start, goal, num_keys, axis=0), 
        constraint=lambda x: constrain_start_goal(x, start, goal)
        )
    opt = tf.optimizers.Adam(learning_rate=0.01)
    for idx in range(steps):
        opt.minimize(lambda: cost(keyframes), var_list=[keyframes])

    return cost(keyframes)

@tf.function
def V_fast(start, goal):
    return cost(tf.linspace(start, goal, 3, axis=0))

In [10]:
@tf.function
def P_goals(goal, num_goals=3):
    return 1/num_goals

@tf.function
def prop_goal(t, start, goal, keyframes, P_goals):
    current_pos = trajectory(t, keyframes)
    numerator = tf.math.exp(V_fast(start, goal) - V_fast(current_pos, goal))
    denominator = ( 
        tf.math.exp(V_fast(start, red) - V_fast(current_pos, red)) + 
        tf.math.exp(V_fast(start, blue) - V_fast(current_pos, blue)) + 
        tf.math.exp(V_fast(start, green) - V_fast(current_pos, green))
    )
    return numerator / denominator * P_goals(goal)

@tf.function
def normalization(t, T=1):
    return T - t

@tf.function
def legibility(start, goal, keyframes):
    numerator = integrate(0, 1, lambda t: prop_goal(t, start, goal, keyframes, P_goals)*normalization(t))
    denominator = integrate(0, 1, lambda t: normalization(t))
    return numerator / denominator

In [11]:
def keyframe_constraint(keyframes):
    keyframes = constrain_start_goal(keyframes, start, goal)
    keyframes = constraint_to_joint_space(keyframes, min_angles, max_angles)
    return keyframes

In [12]:
keyframes = tf.Variable(
    np.load("initial_trajectory.npy").reshape((-1, 7)), 
    dtype=tf.float32,
    constraint=lambda x: keyframe_constraint(x)
)

In [13]:
# compile all graphs

t = tf.cast(tf.linspace(0,1,100), tf.float32)
num_samples=100
a = 0
b = 1
function = lambda t: prop_goal(t, start, goal, keyframes, P_goals)*normalization(t)
positions = tf.cast(a + tf.range(num_samples+1)*(b-a)/num_samples, tf.float32)
values = function(positions)

In [14]:
def target_function(keyframes, start, goal, eta=0.01):
    return - ( legibility(start, goal, keyframes) )

In [15]:
opt = tf.optimizers.Adam(learning_rate=0.01)
old_legibility = legibility(start, goal, keyframes)
current_legibility = tf.Variable(1, dtype=tf.float32)
print(old_legibility)
for idx in range(2):
#while np.abs(current_legibility.numpy() - old_legibility) > 0.01:
    old_legibility = current_legibility.numpy().copy()
    opt.minimize(lambda: target_function(keyframes, start, goal), var_list=[keyframes])
    current_legibility = legibility(start, goal, keyframes)
print(current_legibility)

tf.Tensor(0.21838999, shape=(), dtype=float32)


NameError: name 'min_coord' is not defined

In [49]:
np.save("dragan_trajectory.npy", keyframes.numpy().astype(np.float64))